In [49]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
scope.adc.samples=24400

In [36]:

# function
def setup():
    load(scope, "../c-files/masked-inside-CWLITEARM")
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    rand = [random.randrange(2) for _ in range(64)]
    if op == "ctrl":
        rand = [0 for _ in range(64)]
        p_temp = p0
        k_temp = k0
    elif op == "ctrl_mask":
        p_temp = p0
        k_temp = k0
    elif op == "degen":
        rand = [0 for _ in range(64)]
        p_temp = p1
        k_temp = k0
    elif op == "pmask":
        p_temp = p2
        k_temp = k0
    elif op == "pmask_small":
        p_temp = p3
        k_temp = k0
    elif op == "pmask_large":
        p_temp = p4
        k_temp = k0
    elif op == "kmask_small":
        p_temp = p0
        k_temp = k5
    elif op == "kmask_large":
        p_temp = p0
        k_temp = k6
    else:
        raise "ERROR"

    p_temp = p_temp.puf
    mem = xor(p_temp, s.encode(k_temp))
    posture(target,'k', get_bytes(mem), 16)
    return get_bytes(p_temp + [0] + s.encode(rand))

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    pass

p0 = [random.randrange(2) for _ in range(127)]

k0 = [random.randrange(2) for _ in range(64)] [random.randrange(2) for _ in range(64)]


p1 = p2 = [random.randrange(2) for _ in range(127)]

p3 = list(p0)
for i in random.choices(range(127),k=2):
    p3[i] ^= 1
    
p4 = list(p0)
for i in random.choices(range(127),k=16):
    p4[i] ^= 1

k5 = list(k0)
for i in random.choices(range(64),k=2):
    k5[i] ^= 1

k6 = list(k0)
for i in random.choices(range(64),k=10):
    k6[i] ^= 1

p0 = PUF(p0)
p1 = PUF(p1)
p2 = PUF(p2)
p3 = PUF(p3)
p4 = PUF(p4)

prefix_name = "masked"

env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["ctrl", "ctrl_mask", "degen", "pmask", "pmask_small", "pmask_large", 
                               "kmask_small", "kmask_large"] # operations list
env.subfolder = "masked" # subfolder name
env.TRACES_NUM = 250 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = """masking done inside the component, randomness fed into component, no PUF errors
masking is fed into the CW.
schemes are done followed image sent by itamar


python code:

mem = xor(p_temp, s.encode(k_temp))
posture(target,'k', get_bytes(mem), 16)
return get_bytes(p_temp + [0] + s.encode(rand))


c code:

uint32_t* mem32 = (uint32_t*)mem;
uint32_t* pt32 = (uint32_t*)pt;
for(i = 4; i < 8; i++) {
    pt32[i] ^= mem32[i-4];
    // recd[i] ^= info[i];
}
for(i = 0; i < 4; i++) {
    pt32[i] ^= pt32[i+4];
    // recd[i] ^= info[i];
}
// trigger_low();
for (i = 0; i < length; i++){ // puts input in recd (to decode later)
    recd[i] = (pt[i/8] >> (i%8)) & 1;
}
for (i =0; i<4; i++) {
    pt32[i]=0;
}
"""
s = Scheme() # scheme execution helper
# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(7)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)

# 1
env.title = "Degenerated codeword, mask <= 0; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_1"
split_file(env, file_names, [0, 2], "group1")

# 2
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_2"
split_file(env, file_names, [1, 3], "group2")

# 3
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_3"
env.desc = "|f_{PUF1} - f_{PUF2}| = 2"
split_file(env, file_names, [1, 4], "group3")

# 4
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_4"
env.desc = "|f_{PUF1} - f_{PUF2}| = 16"
split_file(env, file_names, [1, 5], "group4")

# 5
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_5"
env.desc = "|f_{k1} - f_{k2}| = 2"
split_file(env, file_names, [1, 6], "group5")

# 6
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_6"
env.desc = "|f_{k1} - f_{k2}| = 10"
split_file(env, file_names, [1, 7], "group6")

# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13127 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13127 bytes
None
file0 file1 file2 file3 file4 file5 file6 done 2024-07-21 05-24


In [45]:
# masking outside component
# function
def setup():
    print(start_time)
    # load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    load(scope, "../c-files/testsyn-CWLITEARM")
    pass
    
def execute(index, op, t):
    ["ctrl", "ctrl_mask", "degen_eq", "degen_diff", "mask_diff", "mask_eq", 
                               "mask_rand", "synd_diff", "puf_diff"]
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    
    global rand, current_puf
    rand = [random.randrange(2) for _ in range(64)]
    if op == "ctrl":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p0
    elif op == "ctrl_mask":
        p_temp = p0
    elif op == "degen_eq":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p1
    elif op == "degen_diff":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p2
    elif op == "mask_diff":
        p_temp = p3
    elif op == "mask_eq":
        p_temp = p4
    elif op == "mask_rand":
        p5 = xor(p0.puf, s.encode([random.randrange(2) for _ in range(64)]))
        p_temp = PUF(p5)
    elif op == "synd_diff":
        p_temp = p6
    elif op == "puf_diff":
        p_temp = p7
    else:
        raise "a"

    p_temp = p_temp.puf
    posture(target,'k', get_bytes(p_temp), 20)

    current_puf = p_temp
    return get_bytes(p_temp + [0] + s.encode(rand))

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (get_bits(fdb)[:64] != current_puf[-64:]):
        print("WRONG!")
        print(fdb)
        print(xor(get_bits(fdb)[:64],current_puf[-64:]))
        print()
    pass


current_puf = None
s = Scheme() # scheme execution helper

p0 = [random.randrange(2) for _ in range(127)]
p4 = p1 = xor(p0, s.encode([random.randrange(2) for _ in range(64)]))
p3 = p2 = [random.randrange(2) for _ in range(127)]



s6 = s.syndrome(p1)
print(s6)
s6 = list(s6)
s6[5] ^= 126
s6 = tuple(b)
p6 = xor(p1, s.syn2error(s6))

p7 = list(p0)
for i in random.choices(range(127),k=4):
    p7[i] ^= 1
    
p0 = PUF(p0)
p1 = PUF(p1)
p2 = PUF(p2)
p3 = PUF(p3)
p4 = PUF(p4)
p6 = PUF(p6)
p7 = PUF(p7)

prefix_name = "syndrome"

env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["ctrl", "ctrl_mask", "degen_eq", "degen_diff", "mask_diff", "mask_eq", 
                               "mask_rand", "synd_diff", "puf_diff"] # operations list , "set2", "set3"
env.subfolder = "syndrome" # subfolder name
env.TRACES_NUM = 250 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = """masking is done inside the component.
groups taken from itamar image
11 - set vs random same setting as 10 (HD1=HD2)
12 - same as 10 but |HD1 + HD2| = 4 bits
13 - same as 10 but |f1 + f2| = 4 bits

python:

posture(target,'k', get_bytes(p_temp), 20)
return get_bytes(p_temp + [0] + s.encode(rand))

c code:

copying and masking
----- trig high
syndrome();
pompom();
"""


# run loop
start_time = date_str()
print(setup())
time.sleep(0.1)


file_names = [f"file{i}" for i in range(7)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    

# 7
env.title = "Degenerated syndrome, mask <= 0; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_7"
env.desc = "HD_1=HD_2"
split_file(env, file_names, [0, 2], "group7")

# 8
env.title = "Degenerated syndrome, mask <= 0; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_8"
env.desc = "HD_1=/=HD_2"
split_file(env, file_names, [0, 3], "group8")

# 9
env.title = "Degenerated syndrome, mask <= $; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_9"
env.desc = "HD_1=/=HD_2"
split_file(env, file_names, [1, 4], "group9")

# 10
env.title = "Codeword masking, mask <= $ ; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_10"
env.desc = "HD_1 = HD_2"
split_file(env, file_names, [1, 5], "group10")

# 11
env.title = "Codeword masking, mask <= $ ; \{f_{PUF1}\} vs \{f_{PUF2} <= $\}"
env.name = prefix_name + "_11"
env.desc = "HD_1 = HD_2"
split_file(env, file_names, [1, 6], "group11")

# 12
env.title = "Codeword masking, mask <= $ ; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_12"
env.desc = "|HD_1 - HD_2| = 4 (nearby bits)"
split_file(env, file_names, [1, 7], "group12")

# 13
env.title = "Codeword masking, mask <= $ ; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_13"
env.desc = "|f_{PUF1}-f_{PUF2}| = 4"
split_file(env, file_names, [1, 8], "group13")
# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


(85, 119, 113, 107, 22, 127, 95, 61, 53, 18, 0, 43, 12, 51, 122, 79, 107, 15, 62, 2)


AttributeError: 'NoneType' object has no attribute 'controlWrite'

(85, 119, 113, 107, 22, 127, 95, 61, 53, 18, 0, 43, 12, 51, 122, 79, 107, 15, 62, 2)


AttributeError: 'NoneType' object has no attribute 'controlWrite'

In [43]:

# function
def setup():
    # load(scope, "../c-files/masked-inside-CWLITEARM")
    #load(scope, "masked-inside")
    pass

rand = None
glob_k = None
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand, glob_k
    rand = [random.randrange(2) for _ in range(64)]
    if op == "ctrl":
        rand = [0 for _ in range(64)]
        p_temp = p0
        k_temp = k0
    elif op == "ctrl_mask":
        p_temp = p0
        k_temp = k0
    elif op == "degen":
        rand = [0 for _ in range(64)]
        p_temp = p1
        k_temp = k0
    elif op == "pmask":
        p_temp = p2
        k_temp = k0
    elif op == "pmask_small":
        p_temp = p3
        k_temp = k0
    elif op == "pmask_large":
        p_temp = p4
        k_temp = k0
    elif op == "kmask_small":
        p_temp = p0
        k_temp = k5
    elif op == "kmask_large":
        p_temp = p0
        k_temp = k6
    else:
        raise "ERROR"

    p_temp = p_temp.puf
    mem = xor(p_temp, s.encode(k_temp))
    posture(target,'k', get_bytes(mem), 16)
    glob_k = k_temp
    return get_bytes(p_temp + [0] + s.encode(rand))

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (xor(get_bits(fdb)[:64], rand) != glob_k):
        print("WRONG!")
        return True
    pass

p0 = [random.randrange(2) for _ in range(127)]

k0 = [random.randrange(2) for _ in range(64)]


p1 = p2 = [random.randrange(2) for _ in range(127)]

p3 = list(p0)
for i in random.choices(range(127),k=2):
    p3[i] ^= 1
    
p4 = list(p0)
for i in random.choices(range(127),k=16):
    p4[i] ^= 1

k5 = list(k0)
for i in random.choices(range(64),k=2):
    k5[i] ^= 1

k6 = list(k0)
for i in random.choices(range(64),k=10):
    k6[i] ^= 1

p0 = PUF(p0)
p1 = PUF(p1)
p2 = PUF(p2)
p3 = PUF(p3)
p4 = PUF(p4)

prefix_name = "masked"

env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["ctrl", "ctrl_mask", "degen", "pmask", "pmask_small", "pmask_large", 
                               "kmask_small", "kmask_large"] # operations list
env.subfolder = "masked_out" # subfolder name
env.TRACES_NUM = 250 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = "masked out"
env.title = ""
env.name = prefix_name
env.readme = """masking done outside the component, randomness fed into component, no PUF errors
masking is fed into the CW.
schemes are done followed image sent by itamar


python code:

mem = xor(p_temp, s.encode(k_temp))
posture(target,'k', get_bytes(mem), 16)
return get_bytes(p_temp + [0] + s.encode(rand))


c code:

uint32_t* mem32 = (uint32_t*)mem;
uint32_t* pt32 = (uint32_t*)pt;
for(i = 4; i < 8; i++) {
    pt32[i] ^= mem32[i-4];
    // recd[i] ^= info[i];
}
for(i = 0; i < 4; i++) {
    pt32[i] ^= pt32[i+4];
    // recd[i] ^= info[i];
}
// trigger_low();
for (i = 0; i < length; i++){ // puts input in recd (to decode later)
    recd[i] = (pt[i/8] >> (i%8)) & 1;
}
for (i =0; i<4; i++) {
    pt32[i]=0;
}
"""
s = Scheme() # scheme execution helper
# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(7)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)

# 1
env.title = "Degenerated codeword, mask <= 0; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_1"
split_file(env, file_names, [0, 2], "group1")

# 2
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_2"
split_file(env, file_names, [1, 3], "group2")

# 3
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_3"
env.desc = "masked out. |f_{PUF1} - f_{PUF2}| = 2"
split_file(env, file_names, [1, 4], "group3")

# 4
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_4"
env.desc = "masked out. |f_{PUF1} - f_{PUF2}| = 16"
split_file(env, file_names, [1, 5], "group4")

# 5
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_5"
env.desc = "masked out. |f_{k1} - f_{k2}| = 2"
split_file(env, file_names, [1, 6], "group5")

# 6
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_6"
env.desc = "masked out. |f_{k1} - f_{k2}| = 10"
split_file(env, file_names, [1, 7], "group6")

# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


None


KeyboardInterrupt: 

In [21]:
p0 = [random.randrange(2) for _ in range(127)]
p1 = xor(p0, s.encode([random.randrange(2) for _ in range(64)]))
print(s.syndrome(p0))
print(s.syndrome(p1))
a[5]=1
c = s.syndrome(a)
b = list(c)
b[5] = 0
b = tuple(b)
print(s.syn2error(c))
print(s.syn2error(b))

(33, 25, 16, 71, 60, 6, 51, 117, 22, 78, 54, 20, 64, 27, 10, 111, 16, 18, 117, 52)
(33, 25, 16, 71, 60, 6, 51, 117, 22, 78, 54, 20, 64, 27, 10, 111, 16, 18, 117, 52)
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [50]:
# masking outside component
# function
def setup():
    print(start_time)
    # load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    load(scope, "../c-files/testsyn-CWLITEARM")
    pass
    
def execute(index, op, t):
    ["ctrl", "ctrl_mask", "degen_eq", "degen_diff", "mask_diff", "mask_eq", 
                               "mask_rand", "synd_diff", "puf_diff"]
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    
    global rand, current_puf
    rand = [random.randrange(2) for _ in range(64)]
    if op == "ctrl":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p0
    elif op == "ctrl_mask":
        p_temp = p0
    elif op == "degen_eq":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p1
    elif op == "degen_diff":
        rand = [random.randrange(1) for _ in range(64)]
        p_temp = p2
    elif op == "mask_diff":
        p_temp = p3
    elif op == "mask_eq":
        p_temp = p4
    elif op == "mask_rand":
        p5 = xor(p0.puf, s.encode([random.randrange(2) for _ in range(64)]))
        p_temp = PUF(p5)
    elif op == "synd_diff":
        p_temp = p6
    elif op == "puf_diff":
        p_temp = p7
    else:
        raise "a"

    p_temp = p_temp.puf
    posture(target,'k', get_bytes(p_temp), 20)

    current_puf = p_temp
    return get_bytes(p_temp + [0] + s.encode(rand))

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (get_bits(fdb)[:64] != current_puf[-64:]):
        print("WRONG!")
        print(fdb)
        print(xor(get_bits(fdb)[:64],current_puf[-64:]))
        print()
    pass


current_puf = None
s = Scheme() # scheme execution helper

p0 = [random.randrange(2) for _ in range(127)]
p4 = p1 = xor(p0, s.encode([random.randrange(2) for _ in range(64)]))
p3 = p2 = [random.randrange(2) for _ in range(127)]



s6 = s.syndrome(p1)
print(s6)
s6 = list(s6)
s6[5] ^= 126
s6 = tuple(b)
p6 = xor(p1, s.syn2error(s6))

p7 = list(p0)
for i in random.choices(range(127),k=4):
    p7[i] ^= 1
    
p0 = PUF(p0)
p1 = PUF(p1)
p2 = PUF(p2)
p3 = PUF(p3)
p4 = PUF(p4)
p6 = PUF(p6)
p7 = PUF(p7)

prefix_name = "syndrome"

env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["ctrl_mask",
                               "mask_rand", "synd_diff"] # operations list , "set2", "set3"
env.subfolder = "syndrome" # subfolder name
env.TRACES_NUM = 250 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = """masking is done inside the component.
groups taken from itamar image
11 - set vs random same setting as 10 (HD1=HD2)
12 - same as 10 but |HD1 + HD2| = 4 bits
13 - same as 10 but |f1 + f2| = 4 bits

python:

posture(target,'k', get_bytes(p_temp), 20)
return get_bytes(p_temp + [0] + s.encode(rand))

c code:

copying and masking
----- trig high
syndrome();
pompom();
"""


# run loop
start_time = date_str()
print(setup())
time.sleep(0.1)


file_names = [f"file{i}" for i in range(7)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    
# 11
env.title = "Codeword masking, mask <= $ ; \{f_{PUF1}\} vs \{f_{PUF2} <= $\}"
env.name = prefix_name + "_11"
env.desc = "HD_1 = HD_2"
split_file(env, file_names, [0, 1], "group11")

# 12
env.title = "Codeword masking, mask <= $ ; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_12"
env.desc = "|HD_1 - HD_2| = 4 (nearby bits)"
split_file(env, file_names, [0, 2], "group12")


remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


(21, 23, 46, 19, 17, 76, 98, 3, 47, 7, 58, 48, 114, 124, 16, 5, 83, 77, 124, 21)
2024-07-21 08-37
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13343 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13343 bytes
None
file0 file1 file2 file3 file4 file5 file6 done 2024-07-21 08-44


In [44]:
scope.dis()
target.dis()